# 用zkstark来证明爱因斯坦推理题的答案（二）

以下就是先仿照stark101的part1代码，构建一个乘法子群、陪集、用拉格朗日插值解方程，考虑到stark101的解释已经非常完美了，就没必要画蛇添足的增加过多的解释，读者如果不明白，可以去翻一下stark101的教程。https://github.com/starkware-industries/stark101

In [43]:
from field import FieldElement
from polynomial import X, interpolate_poly, Polynomial
from merkle import MerkleTree
from channel import Channel

def extend_to_cyclic_group(x, y):
    x2 = [d for d in range(256)]
    y2 = [0 for d in range(256)]
    i = 0
    for d in x :
        y2[d] = y[i]
        i += 1
        
    return x2, y2

x=[121,105,136,11,120,16,55,125,86,141,20,59,50,34,137,53]
y=[121,105,136,11,120,16,55,125,86,36,129,59,50,118,1,8]
_, y = extend_to_cyclic_group(x, y)
#print(y)


在这里，我们的trace只有16个，但其值域为了完美去重，在前面部分已经选149这个素数作为基，得出的数字值域，然后这里再做一个变形，将其放到$Z_{2^8}$ 中，从二进制表示到素数求余再到这里的$Z_{256}$，有些跳跃，但不难理解他们的等价关系，这里就不展开了。

In [44]:
g = FieldElement.generator() ** (3 * 2 ** 22)
G = [g ** i for i in range(256)]
a = [g ** i for i in y]

f = interpolate_poly(G, a)
#v = f(g**121)
#print(v)
#print(g**121)

拉格朗日插值解方程

In [45]:
w = FieldElement.generator()
h = w ** ((2 ** 30 * 3) // 2048)
H = [h ** i for i in range(2048)]
eval_domain = [w * x for x in H]
f_eval = [f(d) for d in eval_domain]

设定RScode码率$\rho$为$\frac{1}{8}$，因此选一个大小为2048的陪集来验证

In [46]:
f_merkle = MerkleTree(f_eval)
channel = Channel()
channel.send(f_merkle.root)
print(channel.proof)

['send:e57657b5945f7dc18827d470a9d1c52880af5a273a5328a8de2ce0887184e60e']


我们这里不像stark101存在边界约束，并且在stark101里为了简化运算，用了组合多项式的技巧，在本文中不需要，因此后面代码跟stark101里有少许不同，请注意差别。

In [70]:
def next_fri_domain(fri_domain):
    return [x ** 2 for x in fri_domain[:len(fri_domain) // 2]]

def next_fri_polynomial(poly,  beta):
    odd_coefficients = poly.poly[1::2]
    even_coefficients = poly.poly[::2]
    odd = beta * Polynomial(odd_coefficients)
    even = Polynomial(even_coefficients)
    return odd + even

def next_fri_layer(poly, domain, beta):
    next_poly = next_fri_polynomial(poly, beta)
    next_domain = next_fri_domain(domain)
    next_layer = [next_poly(x) for x in next_domain]
    return next_poly, next_domain, next_layer

def FriCommit(cp, domain, cp_eval, cp_merkle, channel):    
    fri_polys = [cp]
    fri_domains = [domain]
    fri_layers = [cp_eval]
    fri_merkles = [cp_merkle]
    while fri_polys[-1].degree() > 0:
        beta = channel.receive_random_field_element()  #获得随机数
        next_poly, next_domain, next_layer = next_fri_layer(fri_polys[-1], fri_domains[-1], beta)
        fri_polys.append(next_poly)
        fri_domains.append(next_domain)
        fri_layers.append(next_layer)
        fri_merkles.append(MerkleTree(next_layer))
        channel.send(fri_merkles[-1].root)   # 发送每一层的merkle 根
    channel.send(str(fri_polys[-1].poly[0]))
    return fri_polys, fri_domains, fri_layers, fri_merkles

fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(f, eval_domain, f_eval, f_merkle, channel)
#print(channel.proof) 

行文至此，已经实现了FRI，对应stark101代码里，我们快速跳过了它的part2部分，上面代码FriCommit对应于part3，下面代码decommit对应于part4。这里解释下channel的作用，这个类模拟了prover跟verifier交互的prover这一侧的动作，（注意verifier侧的动作没提供），因此在commit的时候，需要从verifier那里获得一个随机数，然后发送merkle root给verifier，依次循环，直到最后发送一个常数，如上述代码中， channel.send(str(fri_polys[-1].poly[0])) 所示。

In [63]:
def decommit_on_fri_layers(idx, channel):
    for layer, merkle in zip(fri_layers[:-1], fri_merkles[:-1]):
        length = len(layer)
        idx = idx % length
        sib_idx = (idx + length // 2) % length        
        channel.send(str(layer[idx]))  #对应到verifer的query动作接收相应信息
        channel.send(str(merkle.get_authentication_path(idx)))
        channel.send(str(layer[sib_idx]))
        channel.send(str(merkle.get_authentication_path(sib_idx)))       
    channel.send(str(fri_layers[-1][0]))
    
def decommit_on_query(idx, channel): 
    assert idx < len(f_eval), f'query index: {idx} is out of range. Length of layer: {len(f_eval)}.'
    channel.send(str(f_eval[idx])) 
    channel.send(str(f_merkle.get_authentication_path(idx))) # 
    decommit_on_fri_layers(idx, channel)   
    
def decommit_fri(channel):
    for query in range(3):
        # Get a random index from the verifier and send the corresponding decommitment.
        decommit_on_query(channel.receive_random_int(0, 2047), channel)

需要指出的是，decommit是prover完成的动作，而query是verifier完成的动作，在stark101里并没有提供verifier的实现代码。

In [68]:
import time

#print(channel.proof)
start = time.time()
print("Generating queries and decommitments...")
decommit_fri(channel)
print(f'{time.time() - start}s')
start = time.time()
#print(channel.proof)
print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

Generating queries and decommitments...
0.6715939044952393s
Uncompressed proof length in characters: 448052


至此，我们巧妙的用AIR构造了这个著名的爱因斯坦问题，然后用zkstark的思想对其进行零知识证明，不过我们只完成了prover部分的工作，verifier的工作跟stark101教程里一样，也没有去实现，但原理基本上是清楚的。

最后来一个彩蛋，在stark101的part2中，有这么一个多项式化简的技巧：

$$ \prod_{i=0}^{1023} (x-g^i) = x^{1024}-1 $$ 
不管是PPT、视频都没有解释这个等式是怎么来的，其实原理比较简单，因为方程 $x^{1024}=1=g^0 $ 有1024个解，又因为按照我们对g的定义，刚好$g^i$，$i\in[0..1023]$都满足，所以可以得到上面的等式。
